In [1]:
import numpy as np
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
RESIZE_WIDTH = 192
RESIZE_HEIGHT = 192

In [3]:
class CustomDarker(A.ImageOnlyTransform):
    def __init__(self, max_dark, max_bright, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)  # Pass 'p' to parent
        self.max_dark = max_dark
        self.max_bright = max_bright

    def apply(self, image, **params):
        # Randomly choose a darkening factor between max_dark and max_bright
        factor = 1 + np.random.uniform(self.max_dark, self.max_bright)
        # Apply the factor to darken the image
        image = image * factor
        # Clip values to ensure they stay within valid range [0, 255]
        image = np.clip(image, 0, 255).astype(np.uint8)
        return image


In [ ]:
augmentation_pipeline = A.Compose([
    A.Rotate(limit=10, border_mode=cv2.BORDER_REFLECT, p=1),
    A.RandomScale(scale_limit=0.1, p=0.2),
    A.OneOf([
        CustomDarker(max_dark=-0.4, max_bright=-0.1, p=0.5),
        CustomDarker(max_dark=-0.2, max_bright=0.0, p=0.25),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.25),
    ], p=0.5),
    A.OneOf([
        A.MotionBlur(blur_limit=3, p=0.4),
        A.GaussianBlur(blur_limit=(3, 5), p=0.4),
    ], p=0.2),
    A.ISONoise(p=0.15),
    A.Perspective(p=0.15),
    A.CoarseDropout(num_holes_range=(1, 1), hole_width_range=(16, 48), hole_height_range=(16, 48), p=0.2),
    A.Resize(width=RESIZE_WIDTH, height=RESIZE_HEIGHT, p=1.0),
    ToTensorV2()
])

In [3]:
augmentation_pipeline = A.Compose([
    A.Affine(scale=(0.8, 1.2), translate_percent=(0.1, 0.1), \
             rotate=(-30, 30),  border_mode=cv2.BORDER_CONSTANT, fit_output=False, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.4),
    A.OneOf([
        A.MotionBlur(blur_limit=7, p=0.3),
        A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    ], p=0.3),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.2),
    A.CoarseDropout(num_holes_range=(4, 8), hole_height_range=(32, 32),\
                     hole_width_range=(32, 32), p=0.3),
    A.CLAHE(p=0.2),
    A.Perspective(scale=(0.05, 0.15), p=0.3),
    A.RandomShadow(p=0.2),
    A.ISONoise(p=0.2),
    A.Resize(width=RESIZE_WIDTH, height=RESIZE_HEIGHT, p=1.0),
    ToTensorV2()
])


In [4]:
def augment_image(image, num_augments=6):
    augmented_images = []
    for _ in range(num_augments):
        augmented = augmentation_pipeline(image=image)['image']
        augmented_images.append(augmented)

    return augmented_images

In [ ]:
from matplotlib import gridspec

def preview_augment(image, num_augments=4):
    augmented_images = []
    for _ in range(num_augments):
        augmented = augmentation_pipeline(image=image)['image']
        augmented_images.append(augmented)
    
    return augmented_images

def get_and_preview_augmentation(dataset_path, class_name, sample_count=3):
    import matplotlib.pyplot as plt
    
    image_dir = f"{dataset_path}/{class_name}"
    preview_dir = f"preview_augmented/{class_name}"
    os.makedirs(preview_dir, exist_ok=True)
    
    # Get a list of all images in the directory
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    
    # Take a sample of the images
    if len(image_files) > sample_count:
        image_files = random.sample(image_files, sample_count)
    
    for idx, filename in enumerate(image_files):
        img_path = os.path.join(image_dir, filename)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Resize for preview
        preview_image = cv2.resize(image, (RESIZE_WIDTH, RESIZE_HEIGHT))
        
        # Generate augmented versions
        augmented_images = preview_augment(preview_image, num_augments=4)
        
        # Create a figure for visualization
        fig = plt.figure(figsize=(15, 10))
        gs = gridspec.GridSpec(2, 3)
        
        # Display original image
        ax = plt.subplot(gs[0, 0])
        ax.imshow(preview_image)
        ax.set_title('Original')
        ax.axis('off')
        
        # Display augmented images
        for i, aug_img in enumerate(augmented_images):
            row, col = (i+1) // 3, (i+1) % 3
            ax = plt.subplot(gs[row, col])
            
            # Convert tensor to numpy if needed
            if hasattr(aug_img, 'permute'):
                aug_img = aug_img.permute(1, 2, 0).numpy()
                
            ax.imshow(aug_img)
            ax.set_title(f'Augmented {i+1}')
            ax.axis('off')
        
        # Save the figure
        output_path = os.path.join(preview_dir, f"{filename.split('.')[0]}_preview.png")
        plt.tight_layout()
        plt.savefig(output_path)
        plt.close()
        
        print(f"Saved preview for {filename} to {output_path}")
    
    print(f"Previews generated for {class_name} 🔥")

get_and_preview_augmentation("dataset", "montfleur", sample_count=5)


Saved preview for montfleur_323.jpg to preview_augmented/montfleur\montfleur_323_preview.png
Saved preview for montfleur_265.jpg to preview_augmented/montfleur\montfleur_265_preview.png
Saved preview for montfleur_203.jpg to preview_augmented/montfleur\montfleur_203_preview.png
Saved preview for montfleur_125.jpg to preview_augmented/montfleur\montfleur_125_preview.png
Saved preview for montfleur_262.jpg to preview_augmented/montfleur\montfleur_262_preview.png
Previews generated for montfleur 🔥


In [8]:
import os
import cv2
import asyncio
import psutil
from functools import partial

dataset_path = "dataset"
augmented_path = "augmented"
class_names = ["aura", "minere", "montfleur"]
MAX_WORKERS = min(8, psutil.cpu_count() + 2)
MEMORY_SAFE_MODE = True
CHUNK_SIZE = 2

print(f"⚙️  Config: Workers={MAX_WORKERS}, Chunk={CHUNK_SIZE}, SafeMode={MEMORY_SAFE_MODE}")

semaphore = asyncio.Semaphore(MAX_WORKERS)

async def memory_check():
    """Non-blocking memory monitor"""
    if MEMORY_SAFE_MODE and psutil.virtual_memory().percent > 75:
        print("💤 Memory buffer...")
        await asyncio.sleep(2)
        return True
    return False

def process_image_batch(img_path, num_augments):
    """Process image and return augmented images (let Python's GC handle cleanup)"""
    try:
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        augmented = augment_image(image, num_augments=num_augments)
        # No need to explicitly delete 'image'
        return augmented
    finally:
        cv2.waitKey(1)  # Release OpenCV hooks

async def augment_single_image(filename, image_dir, output_dir, idx_i, num_augments):
    async with semaphore:
        if await memory_check():
            await asyncio.sleep(1)

        img_path = os.path.join(image_dir, filename)
        print(f"📤 Loading {filename}...")
        
        # Process with memory limits
        augmented_images = await asyncio.to_thread(
            partial(process_image_batch, num_augments=num_augments),
            img_path
        )

        # Process in chunks
        write_tasks = []
        base_name = os.path.splitext(filename)[0].split('_')[0]
        
        for idx_j, aug in enumerate(augmented_images):
            if idx_j % CHUNK_SIZE == 0 and idx_j != 0:
                await asyncio.gather(*write_tasks)
                write_tasks.clear()
                await asyncio.sleep(0.2)
            
            output_path = os.path.join(output_dir, f"{base_name}_aug_{idx_i}_{idx_j}.jpg")
            aug_np = aug.permute(1, 2, 0).numpy()
            aug_bgr = cv2.cvtColor(aug_np, cv2.COLOR_RGB2BGR)
            
            write_tasks.append(
                asyncio.to_thread(cv2.imwrite, output_path, aug_bgr)
            )

        # Final flush of remaining tasks
        if write_tasks:
            await asyncio.gather(*write_tasks)

        # Let variables go out of scope naturally
        await asyncio.sleep(0.2)

async def process_augment_class(class_name):
    image_dir = os.path.join(dataset_path, class_name)
    output_dir = os.path.join(augmented_path, class_name)
    os.makedirs(output_dir, exist_ok=True)

    filenames = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    print(f"🔥 Starting {class_name} ({len(filenames)} images)")

    # Process in smaller batches
    BATCH_SIZE = MAX_WORKERS * 1  # Reduced multiplier
    for batch_idx in range(0, len(filenames), BATCH_SIZE):
        batch = filenames[batch_idx:batch_idx+BATCH_SIZE]
        
        tasks = [
            augment_single_image(filename, image_dir, output_dir, idx_i, 8)
            for idx_i, filename in enumerate(batch, start=batch_idx)
        ]
        
        await asyncio.gather(*tasks)
        print(f"✅ Completed batch {batch_idx//BATCH_SIZE + 1}")
        await asyncio.sleep(1)

async def main():
    print("🚀 Starting SAFE augmentation...")
    await asyncio.gather(*[process_augment_class(cls) for cls in class_names])
    print("🎯 All done! PC survived! 🎉")

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())


⚙️  Config: Workers=8, Chunk=2, SafeMode=True
🚀 Starting SAFE augmentation...
🔥 Starting aura (330 images)
🔥 Starting minere (330 images)
🔥 Starting montfleur (330 images)
📤 Loading aura_1.jpg...
📤 Loading aura_10.jpg...
📤 Loading aura_100.jpg...
📤 Loading aura_101.jpg...
📤 Loading aura_102.jpg...
📤 Loading aura_103.jpg...
📤 Loading aura_104.jpg...
📤 Loading aura_105.jpg...
📤 Loading minere_1.jpg...
📤 Loading minere_10.jpg...
📤 Loading minere_100.jpg...
📤 Loading minere_101.jpg...
📤 Loading minere_102.jpg...
📤 Loading minere_103.jpg...
📤 Loading minere_104.jpg...
📤 Loading minere_105.jpg...
📤 Loading montfleur_1.jpg...
✅ Completed batch 1
📤 Loading montfleur_10.jpg...
📤 Loading montfleur_100.jpg...
📤 Loading montfleur_101.jpg...
📤 Loading montfleur_102.jpg...
📤 Loading montfleur_103.jpg...
📤 Loading montfleur_104.jpg...
📤 Loading montfleur_105.jpg...
📤 Loading aura_106.jpg...
📤 Loading aura_107.jpg...
📤 Loading aura_108.jpg...
✅ Completed batch 1
📤 Loading aura_109.jpg...
📤 Loading aur

In [9]:
import gc
gc.collect()

16

In [23]:
import tensorflow as tf
import os
import random

def parse_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode image
    img = tf.image.resize(img, [RESIZE_HEIGHT, RESIZE_WIDTH])
    print(f"Resized image shape: {img.shape}")
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = img / 255.0
    return img, label

def prepare_tf_dataset(data_dir="augmented", original_data_dir="original", batch_size=32, seed=42):
    # Process class directories for augmented and original data
    class_names = sorted(os.listdir(data_dir))
    print(f"Found {len(class_names)} classes: {class_names}")
    
    # Prepare augmented dataset (training)
    augmented_image_paths = []
    augmented_labels = []
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        image_paths = [os.path.join(class_dir, filename) 
                       for filename in os.listdir(class_dir) 
                       if filename.endswith('.jpg')]
        augmented_image_paths.extend(image_paths)
        augmented_labels.extend([class_idx] * len(image_paths))
    
    # Prepare original dataset (validation & test)
    original_image_paths = []
    original_labels = []
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(original_data_dir, class_name)
        image_paths = [os.path.join(class_dir, filename) 
                       for filename in os.listdir(class_dir) 
                       if filename.endswith('.jpg')]
        original_image_paths.extend(image_paths)
        original_labels.extend([class_idx] * len(image_paths))
    
    # Shuffle augmented dataset (training)
    combined = list(zip(augmented_image_paths, augmented_labels))
    random.Random(seed).shuffle(combined)
    augmented_image_paths, augmented_labels = zip(*combined)

    # Shuffle original dataset (validation & test)
    combined_original = list(zip(original_image_paths, original_labels))
    random.Random(seed).shuffle(combined_original)
    original_image_paths, original_labels = zip(*combined_original)
    
    # Use all augmented images for training
    train_paths, train_labels = list(augmented_image_paths[:]), list(augmented_labels[:])
    
    # Fix constant length for validation and test
    start_idx = random.randint(0, 990 - 792 - 1)
    val_paths, val_labels = list(original_image_paths[start_idx:start_idx + 792]), list(original_labels[start_idx:start_idx + 792])

    start_idx = random.randint(0, 990 - 792 - 1)
    test_paths, test_labels = list(original_image_paths[start_idx:start_idx + 792]), list(original_labels[start_idx:start_idx + 792])

    # Create datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

    # Prepare datasets
    train_dataset = (
                        train_dataset
                        .map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                        .batch(batch_size)
                        .prefetch(tf.data.experimental.AUTOTUNE)
                    )
    
    val_dataset =   (
                        val_dataset
                        .map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                        .batch(batch_size)
                        .prefetch(tf.data.experimental.AUTOTUNE)
                    )
    
    test_dataset =  (
                        test_dataset
                        .map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                        .batch(batch_size)
                        .prefetch(tf.data.experimental.AUTOTUNE)
                    )   
    
    print(f"Traing dataset: {len(train_paths)} images")
    print(f"Validation dataset: {len(val_paths)} images")
    print(f"Test dataset: {len(test_paths)} images")
    print("🔥 Datasets ready!")
    return train_dataset, val_dataset, test_dataset, class_names


In [11]:
import pandas as pd
import numpy as np

def print_dataset_examples(dataset, class_names):
    for images, labels in dataset.take(1):
        print(f"Image batch shape: {images.shape}")
        
        # Flatten the images to 1D arrays and create a DataFrame
        flattened_images = [image.numpy().flatten() for image in images]  # Flatten the images
        labels_as_strings = [class_names[label.numpy()] for label in labels]  # Convert labels to class names
        
        # Create a DataFrame
        df = pd.DataFrame(flattened_images)
        df['label'] = labels_as_strings  # Add class labels to the DataFrame
        
        # Display the DataFrame
        print("Dataset Examples (Flattened images and labels):")
        print(df.head())  # Display the first 5 rows of the DataFrame



In [22]:
BATCH_SIZE = 8
SEED = 42

train_dataset, val_dataset, test_dataset, class_names = prepare_tf_dataset(
    data_dir="augmented",
    original_data_dir="dataset",
    batch_size=BATCH_SIZE,
    seed=SEED
)


Found 3 classes: ['aura', 'minere', 'montfleur']


ValueError: Unbatching a tensor is only supported for rank >= 1

In [15]:
print_dataset_examples(train_dataset, class_names)
print_dataset_examples(test_dataset, class_names)

Image batch shape: (8, 192, 192, 3)
Dataset Examples (Flattened images and labels):
          0         1         2         3         4         5         6  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.772549  0.611765  0.407843  0.843137  0.682353  0.478431  0.784314   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7         8         9  ...    110583    110584    110585    110586  \
0  0.000000  0.000000  0.000000  ...  0.396078  0.403922  0.384314  0.439216   
1  0.000000  0.000000  0.000000  ...  0.466667  0.423529  0.337255  0.490196   
2  0.623529  0.419608  0.835294  ...  0.568627  0.537255  0.486275  0.364706   
3  0.000000  0.000000  0.000000  ...  0.466667  0.482353  0.478431  0.647059   
4  0.000000  0.000000  0.000000  ...  0.552941  0.592157  0.60000

In [5]:
print(f"Classes: {class_names}")

for images, labels in train_dataset.take(1):
    print(f"Shape of train images: {images.shape}")
    print(f"Shape of train labels: {labels.shape}")
    print(f"Images type: {type(images)}")
    print(f"Labels type: {type(labels)}")
    print(f"Images dtype: {images.dtype}")
    print(f"Labels dtype: {labels.dtype}")

for images, labels in test_dataset.take(1):
    print(f"Shape of test images: {images.shape}")
    print(f"Shape of test labels: {labels.shape}")
    print(f"Images type: {type(images)}")
    print(f"Labels type: {type(labels)}")
    print(f"Images dtype: {images.dtype}")
    print(f"Labels dtype: {labels.dtype}")

Classes: ['aura', 'minere', 'montfleur']
Shape of train images: (8, 192, 192, 3)
Shape of train labels: (8,)
Images type: <class 'tensorflow.python.framework.ops.EagerTensor'>
Labels type: <class 'tensorflow.python.framework.ops.EagerTensor'>
Images dtype: <dtype: 'float32'>
Labels dtype: <dtype: 'int32'>
Shape of test images: (8, 192, 192, 3)
Shape of test labels: (8,)
Images type: <class 'tensorflow.python.framework.ops.EagerTensor'>
Labels type: <class 'tensorflow.python.framework.ops.EagerTensor'>
Images dtype: <dtype: 'float32'>
Labels dtype: <dtype: 'int32'>


In [16]:
import tensorflow as tf

# Check if TensorFlow is using a GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models

learning_rate = 0.001 
epochs = 14

# Build your model
model = models.Sequential([
    layers.Input(shape=(192, 192, 3)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dense(3, activation="softmax")  # Make sure this matches the number of classes
])

# Compile with explicit learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 190, 190, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 95, 95, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 93, 93, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 44, 44, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 20, 20, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │       153,603 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394,435 (1.50 MB)

 Trainable params: 394,435 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[early_stopping]
)


model.evaluate(test_dataset)

# Save the model to a file
model.save("my_model.keras")

Epoch 1/14
792/792 ━━━━━━━━━━━━━━━━━━━━ 89s 109ms/step - accuracy: 0.5452 - loss: 0.8611 - val_accuracy: 0.9141 - val_loss: 0.2826
Epoch 2/14
792/792 ━━━━━━━━━━━━━━━━━━━━ 92s 114ms/step - accuracy: 0.9158 - loss: 0.2585 - val_accuracy: 0.9331 - val_loss: 0.2376
Epoch 3/14
792/792 ━━━━━━━━━━━━━━━━━━━━ 92s 114ms/step - accuracy: 0.9434 - loss: 0.1593 - val_accuracy: 0.9457 - val_loss: 0.1630
Epoch 4/14
792/792 ━━━━━━━━━━━━━━━━━━━━ 92s 115ms/step - accuracy: 0.9518 - loss: 0.1439 - val_accuracy: 0.9558 - val_loss: 0.1711
Epoch 5/14
792/792 ━━━━━━━━━━━━━━━━━━━━ 90s 111ms/step - accuracy: 0.9721 - loss: 0.0827 - val_accuracy: 0.9621 - val_loss: 0.1186
Epoch 6/14
262/792 ━━━━━━━━━━━━━━━━━━━━ 52s 100ms/step - accuracy: 0.9791 - loss: 0.0625

KeyboardInterrupt: 

In [30]:
model = keras.models.load_model("my_model.keras")

In [32]:
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Assuming you have already trained your model and have the test_dataset ready
# Make predictions on the test dataset
y_true = []
y_pred = []

# Iterate over the test dataset and make predictions
for step, (images, labels) in enumerate(test_dataset):
    print(f"Step {step+1}: Processing batch")

    # True labels (ground truth)
    y_true.extend(labels.numpy())
    print(f"True labels (y_true): {labels.numpy()}")

    # Model predictions
    predictions = model(images, training=False)
    print(f"Model predictions (raw): {predictions.numpy()}")

    # Get the class with the highest score
    predicted_labels = tf.argmax(predictions, axis=-1)
    print(f"Predicted labels (argmax): {predicted_labels.numpy()}")

    # Store predicted labels
    y_pred.extend(predicted_labels.numpy())

# Calculate Precision, Recall, and F1 score using sklearn
precision = precision_score(y_true, y_pred, average='macro')  # Use 'micro', 'macro', or 'weighted' depending on your needs
recall = recall_score(y_true, y_pred, average='macro')  # Use 'micro', 'macro', or 'weighted' depending on your needs
f1 = f1_score(y_true, y_pred, average='macro')  # Calculate F1 score

# Print the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Step 1: Processing batch
True labels (y_true): [1 1 2 0 1 2 1 0]
Model predictions (raw): [[7.0484357e-13 1.0000000e+00 4.8201009e-17]
 [4.9685827e-03 9.8330194e-01 1.1729445e-02]
 [8.0909529e-10 4.1912432e-10 1.0000000e+00]
 [1.0000000e+00 1.4456797e-31 3.4326546e-22]
 [3.8440190e-02 9.6155161e-01 8.1420139e-06]
 [1.0287347e-04 4.9890131e-01 5.0099587e-01]
 [2.1652959e-07 9.9998236e-01 1.7415907e-05]
 [1.0000000e+00 6.6264993e-35 1.8109707e-18]]
Predicted labels (argmax): [1 1 2 0 1 2 1 0]
Step 2: Processing batch
True labels (y_true): [2 2 2 0 0 2 2 0]
Model predictions (raw): [[4.26575145e-31 1.15803855e-32 1.00000000e+00]
 [4.59085186e-06 8.86900008e-01 1.13095380e-01]
 [6.21849954e-01 3.41266870e-01 3.68831344e-02]
 [1.00000000e+00 0.00000000e+00 9.61881373e-24]
 [1.14095268e-21 6.14114590e-20 1.00000000e+00]
 [4.85047891e-25 5.90896552e-29 1.00000000e+00]
 [3.36665950e-14 7.78144695e-08 9.99999881e-01]
 [1.00000000e+00 5.07039837e-18 2.08443873e-10]]
Predicted labels (argmax): [2